In [1]:
import platform
print('Python: ' + platform.python_version())

import numpy as np
print('numpy: ' + np.__version__)

import pandas as pd
print('pandas: ' + pd.__version__)

import matplotlib.pyplot as plt
#print('matplotlib: ' + plt.__version__)

import operator

from CubeScience import CoverageModule as cm

# input: a set of data with match results
# output: a table with final standings

def extract_players(my_matches):
    players = []
    for i in my_matches:
        players.append(i[0]["player"])
        players.append(i[1]["player"])
    return list(set(players))

def normalized_score(player_score):
    deals = player_score["won"] + player_score["lost"]
    win_rate =(player_score["won"]+0.0)/deals
    return player_score["points"]*100 + round(win_rate *100, 0)

def get_sorted_list(players, player_points):
    standings = {}
    for i in players:
        score = normalized_score(player_points[i])
        standings[i] = score
    sorted_standings = sorted(standings.items(), key=operator.itemgetter(1), reverse=True)

    return sorted_standings

# Loading matches from .json file

def loadMatches(filename):
    matches = pd.read_json('data/matches/'+filename)
    match_data = []
    for i in matches["matches"]: 
        record = []
        record.append({"player": i[0]["player"], "score": i[0]["score"]})
        record.append({"player": i[1]["player"], "score": i[1]["score"]})
        match_data.append(record)
    return match_data

# Calculates score for given set of matches

def calculateScores(match_data):
    players = extract_players(match_data)

    player_points = {}
    for i in players:
        player_points[i] = {"points": 0, "won": 0, "lost": 0}

    for i in players:
        for j in match_data:
            if j[0]["player"] == i:
                player_points[i]["won"] +=j[0]["score"]
                if j[0]["score"] > j[1]["score"]:
                    player_points[i]["points"] += 3
                player_points[i]["lost"] +=j[1]["score"]
            if j[1]["player"] == i:
                player_points[i]["won"] +=j[1]["score"]
                if j[1]["score"] > j[0]["score"]:
                    player_points[i]["points"] += 3
                player_points[i]["lost"] +=j[0]["score"]
    
    standings = get_sorted_list(players, player_points)

    #for i in standings:
    #    print i[0], player_points[i[0]]
    
    return standings, player_points
        
def calculateAllDraftScoresForYear(number_of_drafts, yy):
    for i in range(1, number_of_drafts+1):
        print('\nDraft number '+str(i))
        tmp = loadMatches('matches_'+str(yy)+'_'+str(i)+'.json')
        calculateScores(tmp)

def getArrayAverage(my_list):
    total = 0
    for i in my_list:
        total+=i
    return round((total+0.0)/len(my_list), 2)
        
        
def calculateAveragePlaceForPlayer(player, number_of_drafts, yy):
    player_places = []
    for i in range(1, number_of_drafts+1):
        tmp = loadMatches('matches_'+str(yy)+'_'+str(i)+'.json')
        players = extract_players(tmp)
        if player in players:
            #print('\nDraft number '+str(i))
            standings, player_points = calculateScores(tmp)
            for i in range(1, len(standings)+1):
                if standings[i-1][0] == player:
                    #print('Place: '+str(i))
                    player_places.append(i)
    avg_place = getArrayAverage(player_places)
    return avg_place
    
def loadAllMatchesFromYear(number_of_drafts, yy):
    match_data = []
    for i in range(1, number_of_drafts+1):
        tmp = loadMatches('matches_'+str(yy)+'_'+str(i)+'.json')
        for j in tmp:
            match_data.append(j)
    return match_data
        
def getAveragePlaceForAllPlayers(number_of_drafts, yy):
    match_data = loadAllMatchesFromYear(8, 17)
    players = extract_players(match_data)
    player_avg_dict = {}
    for i in players:
        avg_place = calculateAveragePlaceForPlayer(i, number_of_drafts, yy)
        player_avg_dict[i] = avg_place
    return player_avg_dict
                            
def players_per_draft(number_of_drafts, yy):
    frequency_dict = {}
    for i in range(1, number_of_drafts+1):
        tmp = loadMatches('matches_'+str(yy)+'_'+str(i)+'.json')
        frequency_dict[str(i)] = len(extract_players(tmp))
    print(frequency_dict)
    

def prepareTableForCoverage(filename):
    table_string = ""
    new_data = loadMatches(filename)
    my_scores, my_points = calculateScores(new_data)
    results = []
    for i in my_scores: 
        results.append(i[0])
    table_string += '|Pl.|Player|Points|W|L|\n'
    table_string += '|---|---|---|---|---|\n'
    
    for i in range(0, len(results)):
        table_string+= '|'+str(i+1)+'|'+results[i]+'|'+str(my_points[results[i]]['points'])+'|'
        table_string+= str(my_points[results[i]]['won'])
        table_string+= '|'+str(my_points[results[i]]['lost'])+'|\n'
    
    my_file = open('score_table.txt', 'w')
    my_file.write(table_string)
    my_file.close()

prepareTableForCoverage('matches_18_10.json')

# Example input below

match_data = [
     [{"player": "CAS", "score": 2}, {"player": "BAR", "score": 1}],
     [{"player": "CAS", "score": 2}, {"player": "KUB", "score": 1}],
     [{"player": "CAS", "score": 2}, {"player": "MAT", "score": 1}],
     [{"player": "BAR", "score": 2}, {"player": "KUB", "score": 1}],
     [{"player": "BAR", "score": 2}, {"player": "MAT", "score": 1}],
     [{"player": "MAT", "score": 2}, {"player": "KUB", "score": 1}] 
]

#calculateScores(match_data)

#calculateAllDraftScoresForYear(8, 17)

#players_per_draft(8, 17)

#calculateAveragePlaceForPlayer('Casc', 8, 17)

#getAveragePlaceForAllPlayers(8, 17)

cov = cm.Coverage()

cov.loadDecksData('decklists_18_10.json')

#cov.prepareDeckDataForCoverage()



Python: 2.7.9
numpy: 1.13.0
pandas: 0.20.2
0        Kuba
1        SirK
2      JacekK
3      TomekJ
4      Bartek
5      Maciej
6    BartekKu
7        Casc
Name: player, dtype: object


In [5]:
# Preparing graphs for games and matches statistics

import graphviz as gv

import functools
graph = functools.partial(gv.Graph, format='svg')
digraph = functools.partial(gv.Digraph, format='svg')

def add_nodes(graph, nodes):
    for n in nodes:
        if isinstance(n, tuple):
            graph.node(n[0], **n[1])
        else:
            graph.node(n)
    return graph

def add_edges(graph, edges):
    for e in edges:
        if isinstance(e[0], tuple):
            graph.edge(*e[0], **e[1])
        else:
            graph.edge(*e)
    return graph

def apply_styles(graph, styles):
    graph.graph_attr.update(
        ('graph' in styles and styles['graph']) or {}
    )
    graph.node_attr.update(
        ('nodes' in styles and styles['nodes']) or {}
    )
    graph.edge_attr.update(
        ('edges' in styles and styles['edges']) or {}
    )
    return graph

styles = {
    'graph': {
        'label': 'Players and games',
        'fontsize': '16',
        'fontcolor': 'yellow',
        'bgcolor': '#333333',
        'rankdir': 'TB',
    },
    'nodes': {
        'fontname': 'Helvetica',
        'shape': 'oval',
        'fontcolor': 'blue',
        'color': 'white',
        'style': 'filled',
        'fillcolor': '#006699',
    },
    'edges': {
        'style': 'solid',
        'color': 'white',
        'arrowhead': 'open',
        'fontname': 'Courier',
        'fontsize': '12',
        'fontcolor': 'white',
    },
}

def generateGraphAllGames(all_matches, mode):
    players = extract_players(all_matches)

    player_nodes = []
    for i in players:
        player_nodes.append((i, {'label': i, 'shape': 'oval', 'fillcolor': 'yellow'}))
    game_edges = []
    if mode == 'games':
        for i in all_matches:
            for j in range(0, i[0]['score']):
                game_edges.append(((i[1]['player'], i[0]['player'])))
            for j in range(0, i[1]['score']):
                game_edges.append(((i[0]['player'], i[1]['player'])))
    if mode == 'matches':
        for i in all_matches:
            if i[0]['score'] > i[1]['score']:
                game_edges.append(((i[1]['player'], i[0]['player'])))
            else:
                game_edges.append(((i[0]['player'], i[1]['player'])))  

    games_graph = add_edges(
        add_nodes(digraph(), player_nodes), game_edges
    )

    games_graph = apply_styles(games_graph, styles)

    games_graph.render('img/'+mode+'graph')

generateGraphAllGames(loadAllMatchesFromYear(8, 17), 'matches')
generateGraphAllGames(loadAllMatchesFromYear(8, 17), 'games')

In [6]:
# Use weights instead of multiple edges

def generateGraphWeightedEdges(all_matches, mode):
    players = extract_players(all_matches)

    player_nodes = []
    for i in players:
        player_nodes.append((i, {'label': i, 'shape': 'oval', 'fillcolor': 'lightgreen'}))

    edges_library = {}
    
    if mode =='games':
        for i in all_matches:
            for j in range(0, i[0]['score']):
                key = i[1]['player'] +' ' +i[0]['player']
                if edges_library.has_key(key):
                    edges_library[key] +=1
                else:
                    edges_library[key] =1
            for j in range(0, i[1]['score']):
                key = i[0]['player'] +' ' +i[1]['player']
                if edges_library.has_key(key):
                    edges_library[key] +=1
                else:
                    edges_library[key] =1   
    
    if mode == 'matches':
        for i in all_matches:      
            if i[0]['score'] > i[1]['score']:
                key = i[1]['player'] +' ' +i[0]['player']
            else:
                key = i[0]['player'] +' ' +i[1]['player']
            if edges_library.has_key(key):
                edges_library[key] +=1
            else:
                edges_library[key] =1

    weighted_edges = []
    for i in edges_library:
        tmp = i.split(' ')
        tmp = (tmp[0], tmp[1])

        weight = str(edges_library[i])
        wlib = { 'label': weight, 'color': 'lightblue'}
        weighted_edges.append((tmp, wlib))
    
    games_graph = add_edges(add_nodes(digraph(), player_nodes), weighted_edges)

    games_graph = apply_styles(games_graph, styles)

    games_graph.render('img/'+mode+'weighted_graph')

all_matches = loadAllMatchesFromYear(8, 17)

generateGraphWeightedEdges(all_matches, 'matches')